In [ ]:
import numpy as np
from pyphysio import EvenlySignal
def hjorth_features(eeg,fs):
    """
    Calculation of EEG features introduced by Hjorth in "B. Hjorth,
    “EEG analysis based on time domain properties,” Electroencephalogr.
    Clin. Neurophysiol., vol. 29, no. 3, pp. 306–310, 1970."

    Parameters
    ----------
    eeg : array,list -> 1-channel eeg signal
    fs : int -> sample frequency

    Returns
    -------
    hjorth : dic -> dictionary containing activity,mobility and complexity parameters

    """
    eeg_sig = EvenlySignal(values = eeg,
                           sampling_freq = fs,
                           signal_type = 'eeg')

    m = np.sqrt(np.var(np.gradient(eeg_sig))/np.var(eeg_sig))
    c = np.sqrt(np.var(np.gradient(np.gradient(eeg_sig)))/np.var(np.gradient(eeg_sig)))/np.sqrt(np.var(np.gradient(eeg_sig))/np.var(eeg_sig))

    return {
        "mobility" : m,
        "complexity": c
           }

In [ ]:
def other_features(eeg):
    spect = pyeeg.spectral_entropy(eeg,Band=[1,4,7,12,30,40],Fs=250)
    svd = pyeeg.svd_entropy(eeg,2,20)
    fis = pyeeg.fisher_info(eeg,2,20)
    ap = pyeeg.ap_entropy(eeg,2,3)
    hfd = pyeeg.hfd(eeg,9)
    psi,rir = pyeeg.bin_power(eeg,Band=[1,4,7,12,30,40],Fs=250)
    return {
        'spectral_entropy':spect,
        'svd_entropy':svd,
        'fisher_information':fis,
        'approximate entropy':ap,
        'Higuchi Fractal Dimension':hfd,
        'PSI_delta':psi[0],
        'PSI_theta':psi[1],
        'PSI_alpha':psi[2],
        'PSI_beta':psi[3],
        'PSI_gamma':psi[4],
        'RIR_delta':rir[0],
        'RIR_theta':rir[1],
        'RIR_alpha':rir[2],
        'RIR_beta':rir[3],
        'RIR_gamma':rir[4],
    }

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import mne
import scipy
import pyeeg

# Define your statistical_features, band_features, hjorth_features, and discrete_wavelet_features functions

# Function to extract EEG features from FIF file
def extract_eeg_features(user_id, output_path, fs=250):
    # Folder containing the .fif files
    folder_path = "C:/Users/rhyth/Downloads/data"

    emotion_labels_path = "C:/Users/rhyth/Downloads/cleaned_data_readings.csv"
    emotion_labels_data = pd.read_csv(emotion_labels_path)

    select_ch = [21,22,32,23,10,27,12,44,35,46,36,57,51,66,69,8,2,121,123,116,111,107,103,97,86,95,91,61,76,82,74]
    ch_names = ['Cz','FP1','AF3','F7','F3','Fz','FC5','FC1','T7','C3','CP5','CP1','P7','P3','PO3','O1','FP2','AF4','F8','F4','FC6','FC2','T8','C4','CP6','CP2','P8','P4','Pz','PO4','O2','Oz']

    file_names = []
    labels = []
    V = []
    A = []
    files = []

    # Iterate through each row in the emotion labels DataFrame
    for index, row in emotion_labels_data.iterrows():
        file_name = row['file_name']
        emotion_label = row['Groups']
        val = row['V']
        aro = row['A']

        n_fft=1751 #length of fft
        file_path = os.path.join(folder_path, f"{emotion_label}", f"{file_name}")

        # Load raw data
        raw = scipy.io.loadmat(file_path)
        raw_data = []
        for i in select_ch:
            raw_data.append(raw['data'][i])
        raw_data = np.asarray(raw_data)
        labels.append(emotion_label)
        V.append(val)
        A.append(aro)
        file_names.append(file_name)
        files.append(raw_data)

    # Perform feature extraction
    for f in range(len(files)):
        d1 = pd.DataFrame([pyeeg.dfa(files[f][i, :]) for i in tqdm(range(files[f].shape[0]), desc='Detrended fluctuation analysis(DFA)', total=files[f].shape[0])])
        d1.columns = ['DFA']
        d2 = pd.DataFrame([pyeeg.hurst(files[f][i, :]) for i in tqdm(range(files[f].shape[0]), desc='Hurst exponent', total=files[f].shape[0])])
        d2.columns = ['Hurst exponent']
        d3 = pd.DataFrame([hjorth_features(files[f][i, :], fs) for i in tqdm(range(files[f].shape[0]), desc='Hjorth features', total=files[f].shape[0])])
        d4 = pd.DataFrame([pyeeg.pfd(files[f][i, :]) for i in tqdm(range(files[f].shape[0]), desc='Petrosian Fractal Dimension (PFD)', total=files[f].shape[0])])
        d4.columns = ['PFD']
        d5 = pd.DataFrame([other_features(files[f][i, :]) for i in tqdm(range(files[f].shape[0]), desc='others', total=files[f].shape[0])])

        # Standardize the features
        d1 = (d1 - d1.mean()) / d1.std()
        d2 = (d2 - d2.mean()) / d2.std()
        d3 = (d3 - d3.mean()) / d3.std()
        d4 = (d4 - d4.mean()) / d4.std()
        d5 = (d5 - d5.mean()) / d5.std()

        # Concatenate all features
        df = pd.concat([d1, d2, d3, d4, d5], axis=1)

        # Save the features to CSV
        df.to_csv(os.path.join(output_path, f'{file_names[f]}_eegfeatures1.csv'), index=False)


# Example usage
if __name__ == "__main__":
    user_id_input = 0
    # user_id_input = input("Enter the user number: ")
    output_directory = "D:/Feature Fusion/"  # Change this to your desired output directory
    os.makedirs(output_directory, exist_ok=True)

    extract_eeg_features(user_id_input, output_directory)
